In [1]:
import os
import pandas as pd
from statsmodels.tsa.arima.model import ARIMA

In [24]:
OUT_DIR = "../data/research-tables"
os.makedirs(OUT_DIR, exist_ok=True)

In [25]:
df = pd.read_csv("../data/processed/costs-clean.csv")

Дослідницьке питання 1

Довгострокова тенденція (по Україні загалом)

In [16]:
ukraine_df = df[df["region"] == "Україна"].sort_values("year")

print("\n--- Long-term trend (Ukraine) ---")
print(ukraine_df[["year", "costs"]])

growth = ukraine_df["costs"].iloc[-1] - ukraine_df["costs"].iloc[0]
print("Total growth (2005–2019):")
growth


--- Long-term trend (Ukraine) ---
     year      costs
284  2005   335753.0
285  2006   427858.0
286  2007   575510.0
287  2008   793630.0
288  2009   813909.0
289  2010   939308.0
290  2011  1143630.0
291  2012  1310584.0
292  2013  1432467.0
293  2014  1485988.0
294  2015  1740943.0
295  2016  2038740.0
296  2017  2621444.0
297  2018  3217183.0
298  2019  3815511.0
Total growth (2005–2019):


np.float64(3479758.0)

In [17]:
ukraine_trend = ukraine_df[["year", "costs"]].copy()
ukraine_trend["yoy_change"] = ukraine_trend["costs"].diff()
ukraine_trend["yoy_pct"] = (ukraine_trend["costs"].pct_change() * 100).round(2)
ukraine_trend.to_csv(f"{OUT_DIR}/q1_ukraine_trend.csv", index=False)

pd.DataFrame([{
    "start_year": int(ukraine_trend["year"].min()),
    "end_year": int(ukraine_trend["year"].max()),
    "start_costs": float(ukraine_trend["costs"].iloc[0]),
    "end_costs": float(ukraine_trend["costs"].iloc[-1]),
    "total_growth": float(growth),
}]).to_csv(f"{OUT_DIR}/q1_ukraine_summary.csv", index=False)

Дослідницьке питання 3

Найбільші та найменші витрати (останній рік)

In [18]:
latest_year = int(df["year"].max())
latest_data = df[df["year"] == latest_year]

max_region = latest_data.loc[latest_data["costs"].idxmax()]
min_region = latest_data.loc[latest_data["costs"].idxmin()]

print("\n=== Highest & Lowest Regions in", latest_year, "===")
print("Max:", max_region["region"], max_region["costs"])
print("Min:", min_region["region"], min_region["costs"])


=== Highest & Lowest Regions in 2019 ===
Max: Україна 3815511.0
Min: м.Севастополь 14020.0


In [19]:
latest_sorted = latest_data.sort_values("costs", ascending=False)
top5 = latest_sorted.head(5)[["region", "year", "costs"]]
bottom5 = latest_sorted.tail(5)[["region", "year", "costs"]]

top5.to_csv(f"{OUT_DIR}/q2_top5_latest_year.csv", index=False)
bottom5.to_csv(f"{OUT_DIR}/q2_bottom5_latest_year.csv", index=False)

pd.DataFrame([
    {"type": "max", "year": latest_year, "region": max_region["region"], "costs": float(max_region["costs"])},
    {"type": "min", "year": latest_year, "region": min_region["region"], "costs": float(min_region["costs"])},
]).to_csv(f"{OUT_DIR}/q2_extremes_latest_year.csv", index=False)


Дослідницьке питання 3

Перевірка кризових років (середні зміни)

In [26]:
df_sorted = df.sort_values(["region", "year"]).copy()
df_sorted["lag"] = df_sorted.groupby("region")["costs"].shift(1)
df_sorted["change"] = df_sorted["costs"] - df_sorted["lag"]

crisis_years = df_sorted[df_sorted["year"].isin([2008, 2009, 2014, 2015])]

print("\n=== Crisis Years Changes ===")
crisis_mean = crisis_years.groupby("year")["change"].mean()
print(crisis_mean)


=== Crisis Years Changes ===
year
2008    15580.000000
2009     1448.500000
2014     6241.214286
2015    18211.071429
Name: change, dtype: float64


In [27]:
crisis_mean.reset_index().to_csv(f"{OUT_DIR}/q3_crisis_mean_change.csv", index=False)

crisis_years[["region", "year", "costs", "lag", "change"]].to_csv(
    f"{OUT_DIR}/q3_crisis_by_region.csv", index=False
)

Прогнозування витрат на утримання робочої сили

In [28]:
series = ukraine_df.set_index("year")["costs"]

In [34]:
model = ARIMA(series, order=(1, 1, 1))
model_fit = model.fit()

C:\all\politeh\3year\DevOps\labor-costs-ai-analytics\.venv\Lib\site-packages\statsmodels\tsa\statespace\sarimax.py:966: UserWarning: Non-stationary starting autoregressive parameters found. Using zeros as starting parameters.
  warn('Non-stationary starting autoregressive parameters'


In [35]:
forecast = model_fit.forecast(steps=1)

print("Forecast for next 1 years:")
print(forecast)

Forecast for next 1 years:
2020    4.413087e+06
dtype: float64


In [36]:
forecast_df = forecast.reset_index()
forecast_df.columns = ["year", "forecast_costs"]
forecast_df.to_csv(f"{OUT_DIR}/arima_ukraine_forecast.csv", index=False)

fitted_df = model_fit.fittedvalues.reset_index()
fitted_df.columns = ["year", "fitted_costs"]
fitted_df.to_csv(f"{OUT_DIR}/arima_ukraine_fitted.csv", index=False)